<a href="https://colab.research.google.com/github/manargani/VOICE-CLONING/blob/main/DEMO2final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
from pathlib import Path

repo_dir = Path("OpenVoice")

if not repo_dir.exists():
    !git clone https://github.com/myshell-ai/OpenVoice
    orig_english_path = Path("OpenVoice/openvoice/text/_orig_english.py")
    english_path = Path("OpenVoice/openvoice/text/english.py")

    english_path.rename(orig_english_path)

    with orig_english_path.open("r") as f:
        data = f.read()
        data = data.replace("unidecode", "anyascii")
        with english_path.open("w") as out_f:
            out_f.write(data)
# append to sys.path so that modules from the repo could be imported
sys.path.append(str(repo_dir))

%pip install -q "librosa>=0.8.1" "wavmark>=0.0.3" "faster-whisper>=0.9.0" "pydub>=0.25.1" "whisper-timestamped>=1.14.2" "tqdm" "inflect>=7.0.0" "eng_to_ipa>=0.0.2" "pypinyin>=0.50.0" \
"cn2an>=0.5.22" "jieba>=0.42.1" "langid>=1.1.6" "gradio>=4.15" "ipywebrtc" "anyascii" "openvino>=2023.3"

Cloning into 'OpenVoice'...
remote: Enumerating objects: 438, done.
remote: Total 438 (delta 0), reused 0 (delta 0), pack-reused 438 (from 1)
Receiving objects: 100% (438/438), 3.84 MiB | 15.02 MiB/s, done.
Resolving deltas: 100% (207/207), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 37.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 3

In [2]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [3]:
CKPT_BASE_PATH = 'checkpoints'

en_suffix = f'{CKPT_BASE_PATH}/base_speakers/EN'
zh_suffix = f'{CKPT_BASE_PATH}/base_speakers/ZH'
converter_suffix = f'{CKPT_BASE_PATH}/converter'

In [4]:
def download_from_hf_hub(filename, local_dir='./'):
    from huggingface_hub import hf_hub_download
    os.makedirs(local_dir, exist_ok=True)
    hf_hub_download(repo_id="myshell-ai/OpenVoice", filename=filename, local_dir=local_dir)

download_from_hf_hub(f'{converter_suffix}/checkpoint.pth')
download_from_hf_hub(f'{converter_suffix}/config.json')
download_from_hf_hub(f'{en_suffix}/checkpoint.pth')
download_from_hf_hub(f'{en_suffix}/config.json')

download_from_hf_hub(f'{en_suffix}/en_default_se.pth')
download_from_hf_hub(f'{en_suffix}/en_style_se.pth')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


checkpoint.pth:   0%|          | 0.00/131M [00:00<?, ?B/s]

checkpoints/converter/config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

checkpoint.pth:   0%|          | 0.00/160M [00:00<?, ?B/s]

checkpoints/base_speakers/EN/config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

en_default_se.pth:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

en_style_se.pth:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

In [5]:
ckpt_converter = 'checkpoints/converter'
device="cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs'

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


(…)BERP_none0.30_mean1.81_std1.81.model.pkl:   0%|          | 0.00/10.0M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/wavmark/__init__.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(resume_path, map_location=torch.device(

Loaded checkpoint 'checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []


In [6]:
pip install gTTS

In [7]:
import os
from gtts import gTTS

# Set the output directory where the audio file will be saved
output_dir = "./"  # Specify your desired output directory

# Text to convert to speech
text_to_convert = (
    "This project is held by Manar GANI under the supervision of Moulay akhloufi at the university of moncton great opportunity with MITACS."
)

# Step 1: Create a gTTS object
tts = gTTS(text=text_to_convert, lang='en')

# Step 2: Save the audio to a file
audio_file_path = f"{output_dir}/openai_source_output.mp3"
tts.save(audio_file_path)

print(f"Audio file saved to: {audio_file_path}")

Audio file saved to: .//openai_source_output.mp3


In [8]:
base_speaker = ".//openai_source_output.mp3"
source_se, audio_name = se_extractor.get_se(base_speaker, tone_color_converter, vad=True)

reference_speaker = '/content/OpenVoice/resources/example_reference.mp3' # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=True)

OpenVoice version: v1


/usr/local/lib/python3.10/dist-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip


[(0.0, 11.544)]
after vad: dur = 11.544


/usr/local/lib/python3.10/dist-packages/torch/functional.py:666: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:873.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


OpenVoice version: v1
[(0.0, 58.8188125)]
after vad: dur = 58.81798185941043


In [9]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=8f30762d4cb06353afa9e8d2065357070849ab420029ff78062bfadfd64a16ce
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [10]:
import os
from gtts import gTTS
from langdetect import detect # Import the langdetect library

# Text samples in different languages
text = [
    "This project is held by Manar GANI under the supervision of Moulay Akhloufi at the University of Moncton, a great opportunity with MITACS.",
    "Este proyecto está a cargo de Manar GANI bajo la supervisión de Moulay Akhloufi en la Universidad de Moncton, una gran oportunidad con MITACS.",
    "Ce projet est dirigé par Manar GANI sous la supervision de Moulay Akhloufi à l'Université de Moncton, une excellente opportunité avec MITACS.",
    "Dieses Projekt wird von Manar GANI unter der Aufsicht von Moulay Akhloufi an der Universität Moncton durchgeführt, eine großartige Gelegenheit mit MITACS.",
    "Questo progetto è svolto da Manar GANI sotto la supervisione di Moulay Akhloufi presso l'Università di Moncton, una grande opportunità con MITACS.",
    "このプロジェクトは、Manar GANI がモンクトン大学で Moulay Akhloufi の監督の下で行っているもので、MITACS との素晴らしい機会です。",
    "Этот проект проводится Манаром ГАНИ под руководством Мулая Ахлуфи в Университете Монктона, отличная возможность с MITACS.",
    "تم تنفيذ هذا المشروع من قبل منار غاني تحت إشراف مولاي أخلوفي في جامعة مونكتون، فرصة رائعة مع MITACS.",
    "这个项目由 Manar GANI 在 Moncton 大学 Moulay Akhloufi 的监督下进行，与 MITACS 的绝佳机会。",
    "यह परियोजना मनार गनी द्वारा मोनक्टन विश्वविद्यालय में मौलाय अख्लूफी के पर्यवेक्षण में आयोजित की गई थी, MITACS के साथ एक महान अवसर।",
    "Este projeto está a cargo de Manar GANI sob a supervisão de Moulay Akhloufi na Universidade de Moncton, uma grande oportunidade com a MITACS.",
    "المشروع هذا تحت إشراف منار غاني وتحت إشراف مولاي أخلوفي في جامعة مونكتون، فرصة عظيمة مع MITACS."
]

output_dir = "./"  # Replace with your desired output directory
src_path = f'{output_dir}/tmp.wav'

# Loop through each text entry and process it
for i, t in enumerate(text):
    detected_lang = detect(t)
    # Create a gTTS object and save the speech to a temporary file
    tts = gTTS(text=t, lang=detect(t))  # lang="auto" automatically detects the language
    tts.save(src_path)

    save_path = f'{output_dir}/output_crosslingual_{i}.wav'

    # Run the tone color converter (assuming this is defined elsewhere)
    encode_message = "@MyShell"

    tone_color_converter.convert(
        audio_src_path=src_path,
        src_se=source_se,
        tgt_se=target_se,
        output_path=save_path,
        message=encode_message
    )

    print(f"Converted speech saved to: {save_path}")


Converted speech saved to: .//output_crosslingual_0.wav
Converted speech saved to: .//output_crosslingual_1.wav
Converted speech saved to: .//output_crosslingual_2.wav
Converted speech saved to: .//output_crosslingual_3.wav
Converted speech saved to: .//output_crosslingual_4.wav
Converted speech saved to: .//output_crosslingual_5.wav
Converted speech saved to: .//output_crosslingual_6.wav
Converted speech saved to: .//output_crosslingual_7.wav
Converted speech saved to: .//output_crosslingual_8.wav
Converted speech saved to: .//output_crosslingual_9.wav
Converted speech saved to: .//output_crosslingual_10.wav
Converted speech saved to: .//output_crosslingual_11.wav


In [13]:
from IPython.display import Audio, display

# File paths for the base and cloned audio
base_audio = f'{output_dir}/openai_source_output.mp3'  # Base speaker audio
cloned_audio = f'{output_dir}/output_crosslingual_0.wav'  # First cloned output
reference_audio = '/content/OpenVoice/resources/example_reference.mp3'  # Reference voice to clone
# Play the base audio (original voice)
print("Playing base speaker audio:")
display(Audio(base_audio))

print("Playing reference speaker audio:")
display(Audio(reference_audio))

# Play the cloned audio (transformed voice)
print("Playing cloned speaker audio:")
display(Audio(cloned_audio))


Playing base speaker audio:


Playing reference speaker audio:


Playing cloned speaker audio:


In [15]:
pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.6 MB/s eta 0:00:00


In [21]:
# Required Libraries
import numpy as np
import librosa
import jiwer
import soundfile as sf  # For reading and writing audio files

# Function to Calculate Mean Opinion Score (MOS)
def calculate_mos(audio_path, reference_path):
    # In this example, we will assume a fixed MOS score.
    # In practice, this should be based on listener ratings.
    fixed_mos_score = 4.0  # Placeholder for a typical good quality score
    return fixed_mos_score

# Function to Calculate Signal-to-Noise Ratio (SNR)
def calculate_snr(clean_audio_path, noisy_audio_path):
    clean_audio, _ = librosa.load(clean_audio_path, sr=None)
    noisy_audio, _ = librosa.load(noisy_audio_path, sr=None)

    # Ensure both signals are the same length
    min_length = min(len(clean_audio), len(noisy_audio))
    clean_audio = clean_audio[:min_length]
    noisy_audio = noisy_audio[:min_length]

    noise = noisy_audio - clean_audio
    snr = 10 * np.log10(np.sum(clean_audio**2) / np.sum(noise**2)) if np.sum(noise**2) > 0 else float('inf')
    return snr

# Function to Calculate Word Error Rate (WER)
def calculate_wer(reference_text, hypothesis_text):
    return jiwer.wer(reference_text, hypothesis_text)
def interpret_results(mos, snr, wer):
    # Interpret MOS
    if mos >= 4.5:
        mos_interpretation = "Excellent quality. Users will likely be very satisfied."
    elif 3.5 <= mos < 4.5:
        mos_interpretation = "Good quality. Users are generally satisfied."
    elif 2.5 <= mos < 3.5:
        mos_interpretation = "Fair quality. There are some issues, but it may be acceptable."
    else:
        mos_interpretation = "Poor quality. Significant improvements are required."

    # Interpret SNR
    if snr > 20:
        snr_interpretation = "Excellent separation between signal and noise."
    elif 10 < snr <= 20:
        snr_interpretation = "Good separation, but some noise is still present."
    elif 0 < snr <= 10:
        snr_interpretation = "Poor separation between signal and noise; improvements needed."
    else:
        snr_interpretation = "Very poor separation; significant noise is present."

    # Interpret WER
    if wer < 0.05:
        wer_interpretation = "Very low error rate; high accuracy."
    elif 0.05 <= wer < 0.15:
        wer_interpretation = "Moderate error rate; acceptable for many applications."
    elif 0.15 <= wer < 0.25:
        wer_interpretation = "High error rate; significant improvements required."
    else:
        wer_interpretation = "Very high error rate; unacceptable performance."

    return mos_interpretation, snr_interpretation, wer_interpretation


# Example usage
if __name__ == "__main__":
    # Define file paths
    generated_audio_path = 'output_crosslingual_0.wav'  # Path to the generated audio
    clean_audio_path = '/content/OpenVoice/resources/example_reference.mp3'  # Path to the clean reference audio
    reference_text = "This project is held by Manar GANI under the supervision of Moulay Akhloufi at the University of Moncton, a great opportunity with MITACS."  # Original text
    hypothesis_text = "This project is held by Manar GANI under the supervision of Moulay Akhloufi at the University of Moncton, a great opportunity with MITACS."  # Hypothetical generated text for comparison

    # Calculate MOS
    mos_score = calculate_mos(generated_audio_path, clean_audio_path)
    print(f"MOS Score: {mos_score}")

    # Calculate SNR
    snr_value = calculate_snr(clean_audio_path, generated_audio_path)
    print(f"SNR Value: {snr_value} dB")

    # Calculate WER
    wer_value = calculate_wer(reference_text, hypothesis_text)
    print(f"WER Value: {wer_value}")
 # Interpret results
    mos_interpretation, snr_interpretation, wer_interpretation = interpret_results(mos_score, snr_value, wer_value)
    print("Interpretation of Results:")
    print(f"MOS Interpretation: {mos_interpretation}")
    print(f"SNR Interpretation: {snr_interpretation}")
    print(f"WER Interpretation: {wer_interpretation}")


MOS Score: 4.0
SNR Value: -2.9067805409431458 dB
WER Value: 0.0
Interpretation of Results:
MOS Interpretation: Good quality. Users are generally satisfied.
SNR Interpretation: Very poor separation; significant noise is present.
WER Interpretation: Very low error rate; high accuracy.
